In [1]:
%load_ext autoreload
%autoreload 2

from kite_trade import *
from time import sleep
from pushbullet import PushBullet
from pywebio.input import *
from pywebio.output import *
from pywebio.session import *

In [2]:
YEAR='22'
WEEK='D15'
PUSHBULLET_ACCESS_TOKEN=''

In [2]:
#Inorder to get encoding token follow this blog https://marketsetup.in/posts/zerodha-login/

In [3]:
enc_token=''

In [4]:
kite=KiteApp(enctoken=enc_token)
pb = PushBullet(PUSHBULLET_ACCESS_TOKEN)

In [5]:
print(kite.ltp(['NSE:NIFTY 50']))

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18614.4}}}
{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18614.4}}


In [6]:
def get_CE_PE_ltp(strike_price):
    ce_tradingsymbol='NFO:NIFTY'+YEAR+WEEK+str(strike_price)+'CE'
    pe_tradingsymbol='NFO:NIFTY'+YEAR+WEEK+str(strike_price)+'PE'
    res= kite.ltp([ce_tradingsymbol,pe_tradingsymbol])
    return res[ce_tradingsymbol]['last_price'],res[pe_tradingsymbol]['last_price']

In [7]:
def get_NIFTY_ltp():
    return kite.ltp(['NSE:NIFTY 50'])['NSE:NIFTY 50']['last_price']
def nearest_strike(ltp, base=50):
    return base * round(ltp/base)
def get_skew_of_nearest_strike():
    strike=nearest_strike(get_NIFTY_ltp())
    ce,pe=get_CE_PE_ltp(strike)
    print (ce,pe)
    skew = abs(ce-pe)/min(ce,pe)
    return strike,skew

In [8]:
def place_order_short_strangle(strike,lots=1):
    ce_tradingsymbol='NIFTY'+YEAR+WEEK+str(strike)+'CE'
    pe_tradingsymbol='NIFTY'+YEAR+WEEK+str(strike)+'PE'
    print(ce_tradingsymbol)
    kite.place_order(variety=kite.VARIETY_REGULAR,
                    exchange=kite.EXCHANGE_NFO,
                    tradingsymbol=ce_tradingsymbol,
                    transaction_type=kite.TRANSACTION_TYPE_SELL,
                    quantity=lots*50,
                    product=kite.PRODUCT_MIS,
                    order_type=kite.ORDER_TYPE_MARKET,
                    tag='TradeViaPython')
    kite.place_order(variety=kite.VARIETY_REGULAR,
                    exchange=kite.EXCHANGE_NFO,
                    tradingsymbol=pe_tradingsymbol,
                    transaction_type=kite.TRANSACTION_TYPE_SELL,
                    quantity=lots*50,
                    product=kite.PRODUCT_MIS,
                    order_type=kite.ORDER_TYPE_MARKET,
                    tag='TradeViaPython')
    

In [9]:
def square_off_short_strangle(strike,lots=1):
    ce_tradingsymbol='NIFTY'+YEAR+WEEK+str(strike)+'CE'
    pe_tradingsymbol='NIFTY'+YEAR+WEEK+str(strike)+'PE'
    print(ce_tradingsymbol)
    kite.place_order(variety=kite.VARIETY_REGULAR,
                    exchange=kite.EXCHANGE_NFO,
                    tradingsymbol=ce_tradingsymbol,
                    transaction_type=kite.TRANSACTION_TYPE_BUY,
                    quantity=lots*50,
                    product=kite.PRODUCT_MIS,
                    order_type=kite.ORDER_TYPE_MARKET,
                    tag='TradeViaPython')
    kite.place_order(variety=kite.VARIETY_REGULAR,
                    exchange=kite.EXCHANGE_NFO,
                    tradingsymbol=pe_tradingsymbol,
                    transaction_type=kite.TRANSACTION_TYPE_BUY,
                    quantity=lots*50,
                    product=kite.PRODUCT_MIS,
                    order_type=kite.ORDER_TYPE_MARKET,
                    tag='TradeViaPython')

In [10]:
def uncle_theta_trade(lots=1):
    while True:
        strike, skew = get_skew_of_nearest_strike()
        print(strike,':', skew)
        if skew < 0.02:
            print('Skew as required')
            place_order_short_strangle(strike,lots)
            push = pb.push_note('UT Trade Placed', 'Strike:{}'.format(strike))
            break
        sleep(2)

In [11]:
def set_csl(strike,purchase_price,lots=1):
    current_low = purchase_price
    csl=purchase_price*1.3
    print('Initial Stop Loss Set to {}'.format(csl))
    while True:
        ce_ltp,pe_ltp=get_CE_PE_ltp(strike)
        current_price=ce_ltp+pe_ltp
        print(current_price)
        if current_price > csl:
            square_off_short_strangle(strike,lots)
            push = pb.push_note('Stop Loss Hit', 'CSL Stop loss Hit')
            break
        else:
            change = current_low-current_price
            perc_change=(change/float(current_low))*100
            if perc_change > 2:
                current_low = current_price
                csl = current_price*1.3
                push = pb.push_note('Trailing Stop Loss to: {}'.format(csl), 'Stop loss Change')
                print('Trailing Stop Loss to: {}'.format(csl))
        sleep(30)

In [12]:
get_skew_of_nearest_strike()

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18614.4}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518600CE': {'instrument_token': 12109058, 'last_price': 104.1}, 'NFO:NIFTY22D1518600PE': {'instrument_token': 12109314, 'last_price': 30.6}}}
104.1 30.6


(18600, 2.4019607843137254)

In [ ]:
try:
    uncle_theta_trade(lots=10)
except ConnectionError as e:
    print(e)

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18641.1}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 31}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 43.35}}}
31 43.35
18650 : 0.3983870967741936
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18641.9}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 32.35}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 41.95}}}
32.35 41.95
18650 : 0.2967542503863988
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18642.25}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 32.9}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 42.3}}}
32.9 42.3
18650 : 0.2857142857142857
{'status': 'success', 'da

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18643.9}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 34.05}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 41.7}}}
34.05 41.7
18650 : 0.22466960352422927
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18643.9}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 34}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 41.7}}}
34 41.7
18650 : 0.2264705882352942
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18645.5}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 34.15}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 40.75}}}
34.15 40.75
18650 : 0.19326500732064428
{'status': 'success', 'd

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18641.5}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 32.05}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 41.5}}}
32.05 41.5
18650 : 0.294851794071763
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18641.5}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 31.35}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 41.85}}}
31.35 41.85
18650 : 0.33492822966507174
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18640.35}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 31.2}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 42.5}}}
31.2 42.5
18650 : 0.3621794871794872
{'status': 'success',

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18638.95}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.3}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 41.65}}}
30.3 41.65
18650 : 0.3745874587458745
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18637.45}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.5}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 41.05}}}
30.5 41.05
18650 : 0.3459016393442622
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18637.3}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.15}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 40.7}}}
30.15 40.7
18650 : 0.349917081260365
{'status': 'success',

{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.75}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 38.95}}}
30.75 38.95
18650 : 0.2666666666666668
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18639.85}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.85}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 38}}}
30.85 38
18650 : 0.23176661264181517
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18639.35}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.8}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 38.1}}}
30.8 38.1
18650 : 0.23701298701298704
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18638.8}}}
{'status': 'success', 

...

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18640.3}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.45}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 37.05}}}
30.45 37.05
18650 : 0.21674876847290633
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18642.5}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.35}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 36.9}}}
30.35 36.9
18650 : 0.215815485996705
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18641.1}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.6}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 36.95}}}
30.6 36.95
18650 : 0.20751633986928109
{'status': 'success

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18641.6}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.5}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 37.3}}}
30.5 37.3
18650 : 0.22295081967213107
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18640.2}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.15}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 37.85}}}
30.15 37.85
18650 : 0.25538971807628535
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18640.05}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 30.2}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 37.85}}}
30.2 37.85
18650 : 0.25331125827814577
{'status': 'succes

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18639.85}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 29.75}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 38}}}
29.75 38
18650 : 0.2773109243697479
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18637.75}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 28.55}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 39.6}}}
28.55 39.6
18650 : 0.38704028021015763
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18638.65}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 29.05}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 39}}}
29.05 39
18650 : 0.34251290877796897
{'status': 'success', 'da

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18638.85}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 28.55}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 39.3}}}
28.55 39.3
18650 : 0.37653239929947446
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18638.2}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 28.9}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 39.35}}}
28.9 39.35
18650 : 0.3615916955017302
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18637.35}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518650CE': {'instrument_token': 12109570, 'last_price': 28.35}, 'NFO:NIFTY22D1518650PE': {'instrument_token': 12109826, 'last_price': 39.5}}}
28.35 39.5
18650 : 0.39329805996472655
{'status': 'succes

{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18624.4}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518600CE': {'instrument_token': 12109058, 'last_price': 51}, 'NFO:NIFTY22D1518600PE': {'instrument_token': 12109314, 'last_price': 21.8}}}
51 21.8
18600 : 1.3394495412844036
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18624.15}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518600CE': {'instrument_token': 12109058, 'last_price': 50.35}, 'NFO:NIFTY22D1518600PE': {'instrument_token': 12109314, 'last_price': 22.65}}}
50.35 22.65
18600 : 1.2229580573951437
{'status': 'success', 'data': {'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 18624.4}}}
{'status': 'success', 'data': {'NFO:NIFTY22D1518600CE': {'instrument_token': 12109058, 'last_price': 50.8}, 'NFO:NIFTY22D1518600PE': {'instrument_token': 12109314, 'last_price': 22.15}}}
50.8 22.15
18600 : 1.2934537246049662
{'status': 'success', 'da

In [ ]:
place_order_short_strangle(18050)

In [ ]:
place_order_short_strangle(18050)

In [17]:
set_csl(18300,55,10)

Initial Stop Loss Set to 77.0
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 58.2}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 11.45}}}
69.65
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 58.65}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 11.35}}}
70.0
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 58.9}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 11.4}}}
70.3
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 58.55}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 11.5}}}
70.05
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 58.25}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 11.5}}}
69.75
{'statu

{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 60.55}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 10.1}}}
70.64999999999999
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 61.7}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 9.9}}}
71.60000000000001
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 61}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 10}}}
71
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 61.4}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 9.95}}}
71.35
{'status': 'success', 'data': {'NFO:NIFTY22NOV18300CE': {'instrument_token': 15715074, 'last_price': 61.65}, 'NFO:NIFTY22NOV18300PE': {'instrument_token': 15715330, 'last_price': 9.85}}}
71.5
{'status': 'success', '